# 3 подхода к регуляризации весов

In [49]:
import torch
import pandas as pd
from tqdm import trange


def get_xor_data():
    x = torch.tensor([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=torch.float32)
    y = torch.tensor([0, 1, 1, 0], dtype=torch.float32)
    return x, y


def get_2_layer_model():
    torch.manual_seed(1) 
    return torch.nn.Sequential(
    torch.nn.Linear(2, 2), torch.nn.Sigmoid(),
    torch.nn.Linear(2, 1), torch.nn.Sigmoid(),
    torch.nn.Flatten(start_dim=0, end_dim=1)
)

x, y = get_xor_data()
print(f'Items: {len(x)}')

# Параметры
n_epochs = 5
lr = 2
weight_decay = 1e-1
loss_fn = torch.nn.MSELoss(reduction='mean')

Items: 4


## 1. Встроенный weight_decay

- [-] Ненаблюдаемо
- [-] Оптимизирует __все__ веса, в том числе сдвиг, параметры батч нормализации и прочие, которые регуляризировать не требуется
- [-] Реализована только для нормы L2
- [+] Встроенный функционал

In [31]:
model = get_2_layer_model()
loss_history = pd.Series(index=range(n_epochs), dtype=float)

# --------------------------------------------------------
optimizer = torch.optim.SGD(
    model.parameters(), lr=lr, weight_decay=weight_decay
)
# --------------------------------------------------------

for i in range(n_epochs):
    pred = model(x)
    loss = loss_fn(pred, y)
    loss_history[i] = loss.item()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(loss_history)

0    0.254772
1    0.250790
2    0.250126
3    0.250023
4    0.250006
dtype: float64


## 2. Добавление слагаемого к функции потерь
- [+] Возможность просмотра графика -> возможность явного регулирования соответствующего веса
- [+] Можно регулировать какие параметры подвержены регуляризации
- [+] Возможен выбор нормы регуляризации
- [-] Требует большее количество операций для дифференцирования

In [48]:
model = get_2_layer_model()
loss_history = pd.DataFrame(
    index=range(n_epochs),
    columns=['mse', 'l2', 'l2_term', 'loss'],
    dtype=float
)
optimizer = torch.optim.SGD(model.parameters(), lr=lr, weight_decay=0)

for i in range(n_epochs):
    pred = model(x)
    loss = loss_fn(pred, y)
    loss_history.mse[i] = loss
    
    # --------------------------------------------------------
    l2_penalty = torch.tensor(0.0)
    for param in model.parameters():
        l2_penalty += param.square().sum()

#     ERROR! [torch -> python -> torch] loses gradients
#     l2_penalty = torch.sum(torch.tensor(
#         [param.square().sum() for param in model.parameters()]
#     ))
    
    l2_term = (weight_decay / 2) * l2_penalty
    loss += l2_term
    
    loss_history.l2[i] = l2_penalty
    loss_history.l2_term[i] = l2_term
    loss_history.loss[i] = loss
    # --------------------------------------------------------
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
print(loss_history)

        mse        l2   l2_term      loss
0  0.254772  1.142317  0.057116  0.311887
1  0.250790  0.700027  0.035001  0.285791
2  0.250126  0.441927  0.022096  0.272223
3  0.250023  0.281584  0.014079  0.264102
4  0.250006  0.179923  0.008996  0.259002


## 3. Прямая модификация градиента

- [-] Нет возможности просмотра графика
- [+] Можно регулировать какие параметры подвержены регуляризации
- [+] Возможен выбор нормы регуляризации
- [+] Эффективная реализация: уже продифференцировано, не требуется возведение в квадрат, только сумма матриц и умножение на скаляр
- [-] Требуется знание, как выглядит градиент

In [33]:
model = get_2_layer_model()
loss_history = pd.Series(index=range(n_epochs), dtype=float)
optimizer = torch.optim.SGD(model.parameters(), lr=lr, weight_decay=0)

for i in range(n_epochs):
    pred = model(x)
    loss = loss_fn(pred, y)
    loss_history[i] = loss.item()
    optimizer.zero_grad()
    loss.backward()
    
    # --------------------------------------------------------
    for param in model.parameters():
        param.grad += param * weight_decay
    # --------------------------------------------------------

    optimizer.step()

print(loss_history)

0    0.254772
1    0.250790
2    0.250126
3    0.250023
4    0.250006
dtype: float64
